<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<br>
<b> Budlong Wings Code: Generating Extended PSF Wings </b><br>
Use LSST pipeline PSF results to build extended PSF wings for model images. <br> <br>

Contact author: Audrey Budlong <br>
Last verified to run: 16 February 2026 <br>

### Notebook Contents:
1. Imports
2. Define Butler
3. Data Selection
4. Create Composite Model Image (LSST Pipelines)
5. Masks
6. Selecting Bright Sources
7. Retrieve PSF of Selected Source
8. Wings Code
   1. Single fit of scaled PSF
   2. Fit Tail Intensity Parameter
   3. Generate Wings
   4. Visualize Results
9. Extending Wings
10. Difference Image Results

### 1. Imports

In [ ]:
import crowdsource
import lsst.afw.display as afw_display
import lsst.geom as geom
import matplotlib.pyplot as plt
import numpy as np

from astropy import units as u
from astropy.nddata import Cutout2D
from astropy.visualization import simple_norm
from crowdsource.psf import SimplePSF
from crowdsource.crowdsource_base import fit_im
from lsst.daf.butler import Butler
from lsst.geom import SpherePoint, Angle, Point2D
from lsst.utils.plotting import (
    get_multiband_plot_colors,
    get_multiband_plot_symbols,
    get_multiband_plot_linestyles,
)
from matplotlib.patches import Circle
from scipy.interpolate import interp1d
from scipy.ndimage import shift

plt.style.use('seaborn-v0_8-colorblind')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

plt.rcParams['font.family'] = 'serif'

filter_colors = get_multiband_plot_colors()
filter_symbols = get_multiband_plot_symbols()
filter_linestyles = get_multiband_plot_linestyles()

afw_display.setDefaultBackend('firefly')
display = afw_display.Display(frame=1)

### 2. Define Butler

In [ ]:
collections = ["LSSTComCam/DP1"            ]
instrument="LSSTComCam"
skymap="lsst_cells_v1"
repo="/repo/dp1"

butler = Butler(repo, instrument=instrument, collections=collections, skymap=skymap)

visit = 2024112600111
ccd = 8

tract = 10464
patch = 46

### 3. Data Selection

In [ ]:
vi = butler.get("visit_image", dataId={"detector": ccd, "visit": visit})

In [ ]:
datareferences = butler.query_datasets("visit_image", where=f"detector={ccd} and visit={visit} and detector=8")
datareferences

In [ ]:
src_catalog = butler.get("source", dataId={"detector": ccd, "visit": visit})
src_catalog = src_catalog[src_catalog['detector']==ccd]
src_catalog

In [ ]:
# lsstSourceCat_flux = src_catalog["calibFlux"]
lsstSourceCat_flux = src_catalog["psfFlux"]

In [ ]:
index = 3
src_catalog["calibFlux"][index], src_catalog["psfFlux"][index]

In [ ]:
lsstSource_mag = (lsstSourceCat_flux*u.nJy).to(u.ABmag)
lsstSource_mag

In [ ]:
visit_summary = butler.get("visit_summary", dataId={"visit": visit, "detector": ccd})

In [ ]:
wcs = visit_summary.find(ccd).wcs

In [ ]:
sky_points = [
    SpherePoint(geom.Angle(ra, geom.degrees),
                geom.Angle(dec, geom.degrees))
    for ra, dec in zip(src_catalog['coord_ra'], src_catalog['coord_dec'])
]

pixel_points = wcs.skyToPixel(sky_points)

In [ ]:
pixel_points_x = [i.getX() for i in pixel_points]
pixel_points_y = [i.getY() for i in pixel_points]

In [ ]:
src_catalog["x"] = pixel_points_x
src_catalog["y"] = pixel_points_y
src_catalog["pixel points"] = pixel_points
src_catalog["magnitude"] = lsstSource_mag

In [ ]:
psf_model = vi.getPsf()
def retrieve_psf(modelPsf, point):
    psf = modelPsf.computeImage(point)
    psf_array = np.array(psf.array)
    return psf_array

psf_list = []
for pixelPoint in pixel_points:
    lsst_psf = retrieve_psf(psf_model, pixelPoint)
    psf_list.append(lsst_psf)

### 4. Create Composite Model Image (LSST Pipelines)

In [ ]:
def compositeImage(points, psfs, fluxes, pvi):
    H, W = pvi.image.array.shape
    composite = np.zeros((H, W), dtype=float)

    psf_h, psf_w = psfs[0].shape
    cy, cx = (psf_h-1) / 2, (psf_w-1) / 2  # keep as float for sub-pixel

    for point, psf, flux in zip(points, psfs, fluxes):
        x = int(point.getX())
        y = int(point.getY())

        # Integer top-left corner in the image
        x0 = int(np.floor(x - cx))
        y0 = int(np.floor(y - cy))

        # Compute the slice in the image
        x1 = max(0, x0)
        y1 = max(0, y0)
        x2 = min(W, x0 + psf_w)
        y2 = min(H, y0 + psf_h)

        # # Corresponding PSF slice
        px1 = x1 - x0
        py1 = y1 - y0
        px2 = px1 + (x2 - x1)
        py2 = py1 + (y2 - y1)

        composite[y1:y2, x1:x2] += psf[py1:py2, px1:px2] * flux

    # Visualization
    norm = simple_norm(composite, stretch='asinh', percent=99.5)
    plt.figure()
    im = plt.imshow(composite, cmap='gray', origin='lower', norm=norm)
    plt.colorbar(im, label="PSF Calib Flux")
    plt.title("LSST Model Image")
    plt.show()

    return composite

In [ ]:
modelImageArray_subpixel = compositeImage(pixel_points, psf_list, lsstSourceCat_flux, vi)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (20,20))

vi_array = vi.image.array
vi_norm = simple_norm(vi_array, stretch='asinh', percent=99.5)

ax[0].imshow(vi_array, cmap='gray', origin='lower', norm=vi_norm)
ax[0].set_title("Visit Image")


model_norm = simple_norm(modelImageArray_subpixel, stretch='asinh', percent=99.5)
ax[1].imshow(modelImageArray_subpixel, cmap='gray', origin='lower', norm=model_norm)
ax[1].set_title("LSST Model Image — Version 1")

difference = vi_array - modelImageArray_subpixel
dfference_norm = simple_norm(difference, stretch='asinh', percent=99.5)

ax[2].imshow(difference, cmap='gray', origin='lower', norm=dfference_norm)
ax[2].set_title("Difference Image");

### 5. Masks

In [ ]:
mask = vi.getMask()

# Get the bit value corresponding to the NOT_DEBLENDED plane
not_deblended_bit = mask.getPlaneBitMask("NOT_DEBLENDED")

# Mask array is integer-valued, one bit per plane
mask_array = mask.getArray()

# Boolean array: True where NOT_DEBLENDED is set
not_deblended = (mask_array & not_deblended_bit) != 0

In [ ]:
### Visualize Mask

fig, ax = plt.subplots(1, 3, figsize = (20,20))

vi_array = vi.image.array
vi_norm = simple_norm(vi_array, stretch='asinh', percent=99.5)

ax[0].imshow(vi_array, cmap='gray', origin='lower', norm=vi_norm)
ax[0].set_title("Visit Image")

model_norm = simple_norm(modelImageArray_subpixel, stretch='asinh', percent=99.5)

ax[1].imshow(modelImageArray_subpixel, cmap='gray', origin='lower', norm=model_norm)
ax[1].set_title("LSST Model Image — Version 1")

difference = vi_array - modelImageArray_subpixel
dfference_norm = simple_norm(difference, stretch='asinh', percent=99.5)

ax[2].imshow(difference, cmap='gray', origin='lower', norm=dfference_norm)
ax[2].set_title("Difference Image")

from matplotlib.colors import ListedColormap
masked = np.ma.masked_where(~not_deblended, np.ones_like(not_deblended))
cmap = ListedColormap([[1, 0, 0, 1]])  # solid red
cmap.set_bad(alpha=0)
ax[2].imshow(masked, origin="lower", cmap=cmap)

### 6. Selecting Bright Sources

In [ ]:
src_catalog_flux_cap_high = src_catalog[src_catalog["calibFlux"]>=lsstSourceCat_flux.max()/2]

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (20,20))

vi_array = vi.image.array
vi_norm = simple_norm(vi_array, stretch='asinh', percent=99.5)

ax[0].imshow(vi_array, cmap='gray', origin='lower', norm=vi_norm)
ax[0].set_title("Visit Image")


model_norm = simple_norm(modelImageArray_subpixel, stretch='asinh', percent=99.5)
ax[1].imshow(modelImageArray_subpixel, cmap='gray', origin='lower', norm=model_norm)
ax[1].set_title("LSST Model Image — Version 1")

difference = vi_array - modelImageArray_subpixel
dfference_norm = simple_norm(difference, stretch='asinh', percent=99.5)

ax[2].imshow(difference, cmap='gray', origin='lower', norm=dfference_norm)
ax[2].set_title("Difference Image");

for x, y in zip(src_catalog_flux_cap_high["x"], src_catalog_flux_cap_high["y"]):
    ax[2].scatter(x, y, facecolor='none', edgecolor='blue', marker='o', s=40)

masked = np.ma.masked_where(~not_deblended, np.ones_like(not_deblended))
cmap = ListedColormap([[1, 0, 0, 1]])  # solid red
cmap.set_bad(alpha=0)
ax[2].imshow(masked, origin="lower", cmap=cmap)

In [ ]:
selectedSource = src_catalog_flux_cap_high[src_catalog_flux_cap_high["calibFlux"]==max(src_catalog_flux_cap_high["calibFlux"])]

In [ ]:
src_catalog_flux_cap_high.sort("calibFlux", reverse=True)

In [ ]:
selectedSource = src_catalog_flux_cap_high[6]

In [ ]:
selectedSource

In [ ]:
plt.imshow(difference, cmap='gray', origin='lower', norm=dfference_norm)
plt.title("Difference Image");

for x, y in zip(src_catalog_flux_cap_high["x"], src_catalog_flux_cap_high["y"]):
    plt.scatter(x, y, facecolor='none', edgecolor='blue', marker='o', s=40)

masked = np.ma.masked_where(~not_deblended, np.ones_like(not_deblended))
cmap = ListedColormap([[1, 0, 0, 1]])  # solid red
cmap.set_bad(alpha=0)
plt.imshow(masked, origin="lower", cmap=cmap)

plt.scatter(selectedSource["x"], selectedSource["y"], facecolor='none', edgecolor='green', marker='o', s=40)

In [ ]:
selectedSource_spherePoint = SpherePoint(geom.Angle(selectedSource["coord_ra"], geom.degrees), geom.Angle(selectedSource["coord_dec"], geom.degrees))
selectedSource_pixelPoint = wcs.skyToPixel(selectedSource_spherePoint)

selectedSource_calibFlux = selectedSource["calibFlux"]

### 7. Retrieve PSF of Selected Source

In [ ]:
selectedSource_psf = retrieve_psf(psf_model, selectedSource_pixelPoint)

In [ ]:
selectedSource_scaledPsf = selectedSource_psf * selectedSource_calibFlux

In [ ]:
im = plt.imshow(selectedSource_scaledPsf)
plt.colorbar(im)
plt.title("Scaled PSF (PSF * calibFlux)");

### 8. Wings Code

#### I. Single fit of scaled PSF

In [ ]:
def fit_fwhm_from_psf(psf_image, r_max=None):
    """
    Measure the FWHM of a flux-scaled PSF image using a radial profile.

    Parameters
    ----------
    psf_image : 2D numpy array
        PSF image scaled by calibrated flux.
    r_max : float, optional
        Maximum radius (pixels) to consider for the profile.
        If None, uses half the image size.

    Returns
    -------
    fwhm : float
        Full width at half maximum (pixels).
    r_half : float
        Radius at half maximum.
    """
    # Get image shape and center
    ny, nx = psf_image.shape
    cy, cx = (ny - 1) / 2, (nx - 1) / 2

    # Generate a grid of x and y coordinates for each pixel
    y, x = np.indices(psf_image.shape)

    # Find radius
    r = np.sqrt((x - cx)**2 + (y - cy)**2)

    # Flatten arrays - Converts the 2D arrays into 1D vectors 
    # (makes it easier to select all pixels in a radial bin using boolean masks)
    r_flat = r.flatten()
    i_flat = psf_image.flatten()

    # Determine maximum radius if not already specified
    # Ensures we don’t consider pixels outside the PSF image bounds
    # If not specified, uses roughly half the image size
    if r_max is None:
        r_max = min(nx, ny) / 2

    # Radial binning (1 pixel bins)
    # Integer bins between 0 and r_max
    r_bins = np.arange(0, r_max + 1)

    # Center of each bin
    r_centers = 0.5 * (r_bins[:-1] + r_bins[1:])

    # Array to store mean intensity at each radial bin
    prof = np.zeros_like(r_centers)

    # Compute radial profile
    # For each bin: 
        # 1. Select pixels with radius within [r_bin[i], r_bin[i+1])
        # 2. Compute mean intensity
    # Result: prof is the radial profile (1D array of mean intensities as a function of radius)
    for i in range(len(r_centers)):
        mask = (r_flat >= r_bins[i]) & (r_flat < r_bins[i + 1])
        if np.any(mask):
            prof[i] = i_flat[mask].mean()
        else:
            prof[i] = np.nan

    # Drop empty bins (NaNs)
    good = np.isfinite(prof)
    r_centers = r_centers[good]
    prof = prof[good]

    # Peak value (central bin)
    # Find peak intensity and half maximum
    # prof[0] is the center of the PSF, assumed peak
    peak = prof[0]
    # half_max → 50% of peak
    half_max = 0.5 * peak

    # Enforce monotonic decrease in the core
    # (useful if there’s noise or small pixel spikes near the center)
    core = prof <= peak
    r_centers = r_centers[core]
    prof = prof[core]

    # Find radius where profile crosses half max
    # Checks that the profile actually drops below half-maximum
    if np.all(prof > half_max):
        raise RuntimeError("PSF does not drop below half maximum.")

    # Interpolate to find half-maximum radius
    # Uses scipy.interpolate.interp1d inverting the profile (prof → r) 
    # to get a sub-pixel estimate of radius where intensity = half_max
    f = interp1d(prof, r_centers, kind='linear', bounds_error=False)

    #r_half → radius at half-maximum
    r_half = float(f(half_max))

    # FWHM = diameter corresponding to half-maximum
    fwhm = 2.0 * r_half
    return fwhm, r_half


In [ ]:
fwhm, r_half = fit_fwhm_from_psf(selectedSource_scaledPsf)
print(f"FWHM = {fwhm:.3f} pixels")

In [ ]:
psf = selectedSource_scaledPsf
ny, nx = psf.shape

# Center of the PSF (pixel coordinates)
cx = (nx - 1) / 2
cy = (ny - 1) / 2

# Suppose this came from your FWHM fit
fwhm, r_half = fit_fwhm_from_psf(psf)

fig, ax = plt.subplots()
im = ax.imshow(psf, origin='lower')
plt.colorbar(im, ax=ax)

# Overlay half-maximum radius (FWHM / 2)
circle = Circle(
    (cx, cy),
    r_half,
    edgecolor='white',
    facecolor='none',
    linewidth=2,
    linestyle='--'
)

ax.add_patch(circle)
ax.set_title("Scaled PSF (PSF × calibFlux)")

plt.show()


#### II. Fit Tail Intensity Parameter

In [ ]:
def fit_psf_tail_intensity(psf_image, r_half, k=1.25, r_max=None):
    """
    Measure a single-parameter PSF tail intensity using the FWHM scale.

    Parameters
    ----------
    psf_image : 2D numpy array
        Flux-scaled PSF image.
    r_half : float
        Radius at half maximum (from fit_fwhm_from_psf).
    k : float
        Tail starts at k * r_half.
    r_max : float, optional
        Outer radius limit for tail measurement.
        If None, uses half the image size.

    Returns
    -------
    A_tail : float
        Dimensionless tail intensity parameter.
    """
    # Image size and PSF center
    # ny, nx: image dimensions
    # (cx, cy): assumed PSF center (middle of the image)
    ny, nx = psf_image.shape
    cy, cx = (ny - 1) / 2, (nx - 1) / 2

    # Build pixel coordinate grids where x and y are 2D arrays containing the pixel coordinates
    # Each pixel now “knows” its (x, y) location
    y, x = np.indices(psf_image.shape)

    # Compute radial distance from center
    r = np.sqrt((x - cx)**2 + (y - cy)**2)

    # Decide how far out to measure
    # If no outer limit is given, stop at half the image size
    # Prevents including pixels outside the PSF stamp
    if r_max is None:
        r_max = min(nx, ny) / 2

    # Define the tail region mask
    # It selects pixels that are:
        # 1. outside the core (r ≥ k × r_half)
        # 2. inside the usable image area (r ≤ r_max)
        # Basically, this mask isolates the PSF wings
    mask = (r >= k * r_half) & (r <= r_max)

    # Ensures the tail region isn’t empty
    # Prevents silent failures or division by zero
    if not np.any(mask):
        raise RuntimeError("No pixels found in tail region.")

    # Extract tail pixel intensities
    # I_tail is now a 1D array of intensities in the wings
    # These are the pixels we want to characterize
    I_tail = psf_image[mask]

    # Peak intensity (central pixel)
    # Used as the normalization scale
    I_peak = psf_image[int(round(cy)), int(round(cx))]

    # Compute the tail intensity parameter
    # np.mean(I_tail) is the average surface brightness in the wings
    # Divide by I_peak to normalize relative to the core
    # This makes A_tail dimensionless, independent of source flux, and comparable across all PSFs
    A_tail = np.mean(I_tail) / I_peak
    return A_tail

In [ ]:
fwhm, r_half = fit_fwhm_from_psf(selectedSource_scaledPsf)

A_tail = fit_psf_tail_intensity(
    selectedSource_scaledPsf,
    r_half,
    k=1.5
)

print(f"FWHM = {fwhm:.3f} px")
print(f"Tail intensity = {A_tail:.3e}")


#### III. Generate Wings

In [ ]:
def measure_transition_intensity(psf_image, r_transition, dr=1.0):
    """
    Measure the average PSF intensity at a specific transition radius.

    This function computes a single value representing the PSF flux in a thin annulus 
    around the specified radius. It is used to anchor the amplitude of 
    generated PSF wings so that they start smoothly at the correct intensity.

    Parameters
    ----------
    psf_image : 2D numpy array
        Flux-scaled PSF image.
    r_transition : float
        Radius (in pixels) at which the wings are intended to start.
    dr : float, optional
        Width of the radial annulus (in pixels) over which to average the intensity.
        Default is 1.0 pixel.

    Returns
    -------
    I_transition : float
        Mean intensity of the PSF in the annulus around `r_transition`.
        This value is used as the normalization for the wings.
    """
    # ny is the number of rows in psf_image
    # nx is the number of columns in the psf_image
    ny, nx = psf_image.shape

    # (cy, cx) are the coordinates of the center pixel of the PSF
    # (ny-1)/2 ensures correct centering even if the image is an even number of pixels
    cy, cx = (ny - 1) / 2, (nx - 1) / 2

    # x and y are 2D arrays the same size as the psf_image
    # containing the pixel coordinates for every pixel
    y, x = np.indices(psf_image.shape)

    # r is now a 2D array of the same shape as psf_image
    # Each element contains the distance from the center
    # This is how the circular annuli around the PSF center is defined
    r = np.sqrt((x - cx)**2 + (y - cy)**2)

    # Mask is a boolean array: True for pixels within a thin ring around r_transition
    # Width of the ring = dr pixels
    # Only these pixels will be used to compute the average intensity
    mask = (r >= r_transition - dr/2) & (r <= r_transition + dr/2)

    # psf_image[mask] extracts all the pixels inside the annulus
    # np.mean(...) computes the average intensity in that ring
    # Result: a single number — the PSF intensity at the transition radius.
    return np.mean(psf_image[mask])


<center>
Power-law decline used in <code>generate_psf_wings</code>: <br> <br>
$I(r)=I_{transition}(\frac{r}{r_{transition}})^{-\alpha}$ <br><br>
</center>

$I(r)$ <br>
- **Definition:** The intensity of the PSF at radius r in the wings.
- **Units:** Same as the input PSF flux
- **Purpose:** Defines how bright the wings are at any point outside the transition radius.

$I_{transition}$<br>
- **Definition:** The PSF intensity at the transition radius $r_{transition}$
- **How it’s measured:** Using <code>measure_transition_intensity</code> which averages PSF pixels in a thin annulus around $r_{transition}$
- **Purpose:** Anchors the wings so they start at the correct amplitude, avoiding a jump or discontinuity.

$r$ <br>
- **Definition:** Radial distance from the PSF center for the pixel in question.
- **Units:** Pixels (or the same units used in $r_{transition}$)
- **Purpose:** Determines how far from the core each pixel is; needed for the radial falloff.

$r_{transition}$ <br>
- **Definition:** The radius at which the wings start.
- **Units:** Pixels
- **Purpose:** Sets the inner boundary of the wings; inside this radius, the wings are zero.
- **Effect:** Pixels at exactly $r=r_{transition}$ have $I(r)=I_{transition}$

$\alpha$ <br>
- **Definition:** Power-law index of the wings.
- **Units:** Dimensionless
- **Purpose:** Determines how steeply the intensity falls off with radius.

In [ ]:
def generate_psf_wings(psf_image, r_transition, alpha=3.0):
    """
    Generate 2D PSF wings using a power-law function starting at a given radius.

    This function creates a PSF wing array that smoothly extends from the core
    of a PSF. The wings start at the specified transition radius and follow a 
    power-law decline with index `alpha`. The intensity at the transition radius 
    is measured from the input PSF to ensure continuity.

    Parameters
    ----------
    psf_image : 2D numpy array
        Flux-scaled PSF image. Used to measure the intensity at the transition radius.
    r_transition : float
        Radius (in pixels) at which the wings should start.
    alpha : float, optional
        Power-law index describing the falloff of the wings.
        Default is 3.0, corresponding to a typical PSF tail.

    Returns
    -------
    wings : 2D numpy array
        Array of the same shape as `psf_image` containing the wing component.
        Values are zero inside `r_transition` and follow a power-law outside.
    """

    # ny is the number of rows in psf_image
    # nx is the number of columns in the psf_image
    ny, nx = psf_image.shape

    # (cy, cx) are the coordinates of the center pixel of the PSF
    # (ny-1)/2 ensures correct centering even if the image is an even number of pixels
    cy, cx = (ny - 1) / 2, (nx - 1) / 2

    # x and y are 2D arrays the same size as the psf_image
    # containing the pixel coordinates for every pixel
    y, x = np.indices(psf_image.shape)

    # r is now a 2D array of the same shape as psf_image
    # Each element contains the distance from the center
    # This is how the circular annuli around the PSF center is defined
    r = np.sqrt((x - cx)**2 + (y - cy)**2)

    # Measure the intensity at the transition radius
    # Computes the average PSF intensity in a thin annulus around r_transition
    # This ensures the wings start at the correct amplitude and don’t create a discontinuity
    I_transition = measure_transition_intensity(psf_image, r_transition)

    # Initialize the wings array
    # Create a 2D array of zeros the same shape as the PSF
    # Pixels inside the transition radius will remain zero, so the core is untouched
    wings = np.zeros_like(psf_image)

    # Define a mask for pixels outside the transition radius
    # 'mask' is a boolean array
    # True for all pixels outside the transition radius, where wings will exist
    mask = r >= r_transition

    # Assign power-law intensity to the wings
    # For pixels in the wings region, the intensity follows a power-law decline (shown above)
    # Ensures smooth radial falloff starting at the measured intensity
    # Pixels inside r_transition remain zero
    wings[mask] = I_transition * (r[mask] / r_transition)**(-alpha)

    # Return the wings array
    # Output is a 2D array of the same size as psf_image
    # Can be added directly to the core PSF or plotted independently
    return wings


In [ ]:
core_psf = selectedSource_scaledPsf

# choose physically correct transition radius
r_transition = 1.75 * r_half

wings = generate_psf_wings(
    core_psf,
    r_transition,
    alpha=3.0
)

psf_with_wings = core_psf.copy()

mask = wings > 0

psf_with_wings[mask] = wings[mask]


#### IV. Visualize Results

In [ ]:
plt.figure()

row = core_psf.shape[0] // 2

plt.plot(core_psf[row], label="Core")
plt.plot(psf_with_wings[row], label="Core + Wings")

# center column coordinate in pixel units - radius (in pixels) where the wings are intended to start
plt.axvline(core_psf.shape[1]/2 - r_transition, color='red', label='Wing transition radius')
plt.axvline(core_psf.shape[1]/2 + r_transition, color='red')

plt.yscale("log")
plt.xlabel("Pixels")
plt.ylabel("Flux (nJy)")
plt.title("PSF Cross-Section")
plt.legend()
plt.show()


In [ ]:
ny, nx = wings.shape
cy, cx = (ny - 1) / 2, (nx - 1) / 2

row = int(round(cy))

# slices
wings_slice = wings[row, :]
core_slice = core_psf[row, :]
full_slice = psf_with_wings[row, :]

# radius axis centered at PSF center
x = np.arange(nx)
r = x - cx

plt.figure(figsize=(8,5))

plt.plot(r, core_slice, label="Core PSF", linewidth=2, alpha=0.5)
plt.plot(r, wings_slice, label="Wings", linewidth=2, alpha=0.5)
plt.plot(r, full_slice, label="Core + Wings", linewidth=2, alpha=0.5)

plt.yscale("log")

plt.xlabel("Radius from center (pixels)")
plt.ylabel("Flux (nJy)")
plt.title("PSF Cross-Section")

plt.axvline(-r_transition, linestyle="--", color="red", label="Wing transition radius")
plt.axvline(r_transition, linestyle="--", color="red")

plt.legend()
plt.grid(True, alpha=0.3)

plt.show()

In [ ]:
mask = r >= 0

plt.figure(figsize=(8,5))

plt.plot(r[mask], core_slice[mask], label="Core")
plt.plot(r[mask], wings_slice[mask], label="Wings")
plt.plot(r[mask], full_slice[mask], label="Core + Wings")

plt.yscale("log")

plt.axvline(r_transition, linestyle="--", color="red", label="Wing transition radius")

plt.xlabel("Radius from center (pixels)")
plt.ylabel("Flux (nJy)")
plt.title("PSF Half Cross-Section")

plt.legend()
plt.grid(True, alpha=0.3)

plt.show()


In [ ]:
core_norm = simple_norm(core_psf, stretch='log', percent=99.5)
fig, axes = plt.subplots(1, 3, figsize=(20, 5))
core_im = axes[0].imshow(core_psf, origin='lower', norm=core_norm, cmap='Blues')
plt.colorbar(core_im, label="PSF Instant Flux (nJy)", ax=axes[0], fraction=0.046, pad=0.04)
axes[0].set_title("Core PSF: Log Scale")

# wings_norm = simple_norm(wings, stretch='log', percent=99.5)
wings_im = axes[1].imshow(wings, origin='lower', norm=core_norm, cmap='Blues')
plt.colorbar(wings_im, label="PSF Instant Flux (nJy)", ax=axes[1], fraction=0.046, pad=0.04)
axes[1].set_title("Generated Wings: Log Scale")

# full_norm = simple_norm(psf_with_wings, stretch='log', percent=99.5)
full_im = axes[2].imshow(psf_with_wings, origin='lower', norm=core_norm, cmap='Blues')
plt.colorbar(full_im, label="PSF Instant Flux (nJy)", ax=axes[2], fraction=0.046, pad=0.04)
axes[2].set_title("Core + Wings: Log Scale")

plt.show()


### 9. Extending Wings

In [ ]:
def generate_psf_with_smooth_wings(core_psf, r_transition, alpha=3.0, I_min_frac=1e-6, smooth_scale=1.0):
    """
    Generate a PSF with smoothly blended extended wings.

    The wings start at `r_transition` and follow a power-law decline.
    The transition from core to wings is smoothed with a sigmoid function
    to avoid sharp drops.

    Parameters
    ----------
    core_psf : 2D numpy array
        Flux-scaled PSF image (core).
    r_transition : float
        Radius (pixels) at which the wings start.
    alpha : float, optional
        Power-law index for the wings (default: 3.0).
    I_min_frac : float, optional
        Fraction of the transition intensity at which to stop the wings (default: 1e-6).
    smooth_scale : float, optional
        Smoothing scale in pixels for blending core and wings (default: 1.0).

    Returns
    -------
    psf_ext : 2D numpy array
        Core + smoothly extended wings.
    wings_ext : 2D numpy array
        Extended wings only.
    """

    # Measure transition intensity
    # Computes the average intensity of the core PSF at r_transition
    # This ensures the wings start smoothly at the correct amplitude
    I_transition = measure_transition_intensity(core_psf, r_transition)

    # Determine how far the wings should extend
    # Compute max radius where wings fall to I_min_frac
    # r_max is the radius where the wings fall to a fraction I_min_frac of I_transition
    # Formula: I_transition * (r_max / r_transition)^(-alpha) = I_transition * I_min_frac; solves for r_max
    # np.ceil rounds up to nearest integer pixel for array indexing
    r_max = r_transition * (1 / I_min_frac)**(1/alpha)
    r_max = int(np.ceil(r_max))

    # Compute extended array size and center
    # Core PSF may need to expand to fit wings beyond its original size
    # pad_y, pad_x are the number of pixels to add on each side
    ny, nx = core_psf.shape
    pad_y = int(np.ceil(r_max - (ny-1)/2))
    pad_x = int(np.ceil(r_max - (nx-1)/2))

    # ny_ext, nx_ext are used to define the total size of new extended array
    ny_ext, nx_ext = ny + 2*pad_y, nx + 2*pad_x
    
    # cy_ext, cx_ext are the coordinates of the center of the extended array
    cy_ext, cx_ext = (ny_ext-1)/2, (nx_ext-1)/2

    # Create coordinate grids
    # np.indices(...) creates 2D arrays of x and y pixel coordinates
    y, x = np.indices((ny_ext, nx_ext))
    # r is the 2D array of radial distances from the center; used to define the wings and blending
    r = np.sqrt((x - cx_ext)**2 + (y - cy_ext)**2)

    # Generate wings
    # Empty array (all zeros initially)
    wings_ext = np.zeros((ny_ext, nx_ext))

    # Boolean mask for pixels outside r_transition
    mask_wings = r >= r_transition

    # Assign power-law intensity to pixels in the wings region
    # Inside r_transition it remains zero (no wings overlapping the core)
    wings_ext[mask_wings] = I_transition * (r[mask_wings]/r_transition)**(-alpha)

    # Pad core PSF into extended array
    # Create a new array of zeros same size as wings_ext
    psf_core_ext = np.zeros_like(wings_ext)

    # Insert the original core PSF into the center
    # This ensures that the core and wings are in the same array and properly centered
    psf_core_ext[pad_y:pad_y+ny, pad_x:pad_x+nx] = core_psf

    # Smooth radial blending: sigmoid-like function
    # Creates a radial weighting function that smoothly transitions from 0 → 1 around r_transition
    # https://iopscience.iop.org/article/10.3847/1538-3881/ad6a0f
    blend = 1 / (1 + np.exp(-(r - r_transition)/smooth_scale))
    # Inside r_transition: blend ≈ 0 → core dominates
    # Outside r_transition: blend ≈ 1 → wings dominate

    # Combine core and wings
    # Weighted sum of core and wings using the blend mask
    # Produces a smooth, continuous PSF without artificial drops
    psf_ext = (1 - blend) * psf_core_ext + blend * wings_ext

    # psf_ext: full PSF with smooth wings
    # wings_ext: wings only (useful for visualization)
    return psf_ext, wings_ext


In [ ]:
core_psf = selectedSource_scaledPsf
r_transition = 1.75 * r_half
alpha = 3.0
smooth_scale = 2.0  # pixels over which the transition occurs

psf_ext, wings_ext = generate_psf_with_smooth_wings(core_psf, r_transition, alpha, I_min_frac=1e-6, smooth_scale=smooth_scale)

# Central row
row = psf_ext.shape[0] // 2
x_ext = np.arange(psf_ext.shape[1]) - (psf_ext.shape[1]-1)/2

import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
plt.plot(x_ext, psf_ext[row], label="Core + Smooth Wings", linewidth=2)
plt.plot(x_ext, wings_ext[row], label="Wings Only", linewidth=2)
plt.plot(np.arange(core_psf.shape[1]) - (core_psf.shape[1]-1)/2, core_psf[core_psf.shape[0]//2], label="Core PSF", linewidth=2)
plt.yscale("log")
plt.xlabel("Radius (pixels)")
plt.ylabel("Intensity")
plt.title("PSF Cross-Section with Smooth Wings")
plt.legend()
# plt.xlim(-50,50)
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
core_psf = selectedSource_scaledPsf
r_transition = 1.75 * r_half
alpha = 3.0
smooth_scale = 2.0  # pixels over which the transition occurs

psf_ext, wings_ext = generate_psf_with_smooth_wings(core_psf, r_transition, alpha, I_min_frac=1e-6, smooth_scale=smooth_scale)

# Central row
row = psf_ext.shape[0] // 2
x_ext = np.arange(psf_ext.shape[1]) - (psf_ext.shape[1]-1)/2

import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
plt.plot(x_ext, psf_ext[row], label="Core + Smooth Wings", linewidth=2)
plt.plot(x_ext, wings_ext[row], label="Wings Only", linewidth=2)
plt.plot(np.arange(core_psf.shape[1]) - (core_psf.shape[1]-1)/2, core_psf[core_psf.shape[0]//2], label="Core PSF", linewidth=2)
plt.yscale("log")
plt.xlabel("Radius (pixels)")
plt.ylabel("Intensity")
plt.title("PSF Cross-Section with Smooth Wings")
plt.legend()
plt.xlim(-50,50)
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# Use the same normalization for all three images for comparison
norm = simple_norm(core_psf, stretch='log', percent=99.5)

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Core PSF
im0 = axes[0].imshow(core_psf, origin='lower', norm=norm, cmap='viridis')
axes[0].set_title("Core PSF")
plt.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04, label="Flux (nJy)")

# Wings only
im1 = axes[1].imshow(wings_ext, origin='lower', norm=norm, cmap='viridis')
axes[1].set_title("Extended Wings")
plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04, label="Flux (nJy)")

# Core + wings
im2 = axes[2].imshow(psf_ext, origin='lower', norm=norm, cmap='viridis')
axes[2].set_title("Core + Smooth Extended Wings")
plt.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04, label="Flux (nJy)")

for ax in axes:
    ax.set_xlabel("X pixels")
    ax.set_ylabel("Y pixels")

plt.tight_layout()
plt.show()


In [ ]:
# Generate smoothly extended wings
r_transition = 2 * r_half
alpha = 3.0
smooth_scale = 2.0
psf_ext, wings_ext = generate_psf_with_smooth_wings(core_psf, r_transition, alpha, I_min_frac=1e-6, smooth_scale=smooth_scale)

# Same normalization for all images
norm = simple_norm(core_psf, stretch='log', percent=99.5)

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Core PSF
ny, nx = core_psf.shape
extent_core = [-(nx-1)/2, (nx-1)/2, -(ny-1)/2, (ny-1)/2]  # center axes at 0
im0 = axes[0].imshow(core_psf, origin='lower', norm=norm, cmap='Blues', extent=extent_core)
axes[0].set_title("Core PSF")
axes[0].set_xlabel("X (pixels)")
axes[0].set_ylabel("Y (pixels)")
axes[0].set_xlim(-10,10)
axes[0].set_ylim(-10,10)
plt.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04, label="Intensity")

# Extended Wings
ny, nx = wings_ext.shape
extent_wings = [-(nx-1)/2, (nx-1)/2, -(ny-1)/2, (ny-1)/2]  # center axes at 0
im1 = axes[1].imshow(wings_ext, origin='lower', norm=norm, cmap='Blues', extent=extent_wings)
axes[1].set_title("Extended Wings")
axes[1].set_xlabel("X (pixels)")
axes[1].set_ylabel("Y (pixels)")
axes[1].set_xlim(-10,10)
axes[1].set_ylim(-10,10)
plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04, label="Intensity")

# Core + Smooth Extended Wings
ny, nx = psf_ext.shape
extent_full = [-(nx-1)/2, (nx-1)/2, -(ny-1)/2, (ny-1)/2]  # center axes at 0
im2 = axes[2].imshow(psf_ext, origin='lower', norm=norm, cmap='Blues', extent=extent_full)
axes[2].set_title("Core + Smooth Extended Wings")
axes[2].set_xlabel("X (pixels)")
axes[2].set_ylabel("Y (pixels)")
axes[2].set_xlim(-10,10)
axes[2].set_ylim(-10,10)
plt.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04, label="Intensity")

plt.tight_layout()
plt.show()


### 10. Difference Image Results

In [ ]:
# Assuming you already have the visit image array
vi_array = vi.image.array  # 2D array

# Pixel coordinates of your source
x_center, y_center = selectedSource_pixelPoint
x_center = float(x_center)
y_center = float(y_center)

# Size of cutout (square)
cutout_size = 51  # pixels, adjust as needed

# Create the cutout
cutout = Cutout2D(
    vi_array,
    position=(x_center, y_center),
    size=(cutout_size, cutout_size)
)

# Extract cutout data
cutout_data = cutout.data

# Create coordinate grids centered at 0
ny, nx = cutout_data.shape
y_grid, x_grid = np.indices((ny, nx))
x_grid = x_grid - (nx - 1)/2
y_grid = y_grid - (ny - 1)/2


In [ ]:
# Optional: use log stretch for better dynamic range
norm = simple_norm(cutout_data, stretch='log', percent=99.5)

plt.figure(figsize=(6,6))
im = plt.imshow(cutout_data, origin='lower', norm=norm, cmap='viridis',
                extent=[x_grid.min(), x_grid.max(), y_grid.min(), y_grid.max()])
plt.colorbar(im, label="Pixel Value", fraction=0.046, pad=0.04)
plt.xlabel("x (pixels, centered)")
plt.ylabel("y (pixels, centered)")
plt.title("Visit Image Cutout")
plt.grid(False)
plt.show()


In [ ]:
# Determine image size
ny, nx = selectedSource_scaledPsf.shape

lsst_norm = simple_norm(selectedSource_scaledPsf, stretch='log', percent=99.5)

# Define x and y coordinates so x=0 is at the center
x = np.arange(-nx//2, nx//2)
y = np.arange(-ny//2, ny//2)

# Plot using extent
im = plt.imshow(
    selectedSource_scaledPsf,
    extent=[x[0], x[-1], y[0], y[-1]],
    origin='lower',  # so y increases upwards
    norm=lsst_norm
)
plt.colorbar(im)
plt.title("LSST PSF (scaled)")
plt.xlabel("x (pixels, centered)")
plt.ylabel("y (pixels, centered)")
plt.show()

In [ ]:
# Norms for dynamic range
cutout_norm = simple_norm(cutout_data, stretch='log', percent=99.5)
psf_norm = simple_norm(selectedSource_scaledPsf, stretch='log', percent=99.5)

# Determine PSF coordinates for centering
ny, nx = selectedSource_scaledPsf.shape
x_psf = np.arange(-nx//2, nx//2)
y_psf = np.arange(-ny//2, ny//2)

# Create figure with 2 subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns

# ----- Subplot 1: Visit Image Cutout -----
im1 = axes[0].imshow(
    cutout_data,
    origin='lower',
    norm=cutout_norm,
    cmap='viridis',
    extent=[x_grid.min(), x_grid.max(), y_grid.min(), y_grid.max()]
)
axes[0].set_title("Visit Image Cutout")
axes[0].set_xlabel("x (pixels, centered)")
axes[0].set_ylabel("y (pixels, centered)")
axes[0].grid(False)
fig.colorbar(im1, ax=axes[0], label="Flux (nJy)", fraction=0.046, pad=0.04)

# ----- Subplot 2: LSST PSF -----
im2 = axes[1].imshow(
    selectedSource_scaledPsf,
    extent=[x_psf[0], x_psf[-1], y_psf[0], y_psf[-1]],
    origin='lower',
    norm=psf_norm,
    cmap='viridis'
)
axes[1].set_title("LSST PSF (scaled)")
axes[1].set_xlabel("x (pixels, centered)")
axes[1].set_ylabel("y (pixels, centered)")
fig.colorbar(im2, ax=axes[1], label="Flux (nJy)", fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()


In [ ]:
# Norms for dynamic range
cutout_norm = simple_norm(cutout_data, stretch='log', percent=99.5)
psf_norm = simple_norm(selectedSource_scaledPsf, stretch='log', percent=99.5)

# Determine PSF coordinates for centering
ny, nx = selectedSource_scaledPsf.shape
x_psf = np.arange(-nx//2, nx//2)
y_psf = np.arange(-ny//2, ny//2)

# Create figure with 2 subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns

# ----- Subplot 1: Visit Image Cutout -----
im1 = axes[0].imshow(
    cutout_data,
    origin='lower',
    norm=cutout_norm,
    cmap='viridis',
    extent=[x_grid.min(), x_grid.max(), y_grid.min(), y_grid.max()]
)
axes[0].set_title("Visit Image Cutout")
axes[0].set_xlabel("x (pixels, centered)")
axes[0].set_ylabel("y (pixels, centered)")
axes[0].grid(False)
axes[0].set_xlim(-11, 11)
axes[0].set_ylim(-11, 11)
fig.colorbar(im1, ax=axes[0], label="Flux (nJy)", fraction=0.046, pad=0.04)

# ----- Subplot 2: LSST PSF -----
im2 = axes[1].imshow(
    selectedSource_scaledPsf,
    extent=[x_psf[0], x_psf[-1], y_psf[0], y_psf[-1]],
    origin='lower',
    norm=psf_norm,
    cmap='viridis'
)
axes[1].set_title("LSST PSF (scaled)")
axes[1].set_xlabel("x (pixels, centered)")
axes[1].set_ylabel("y (pixels, centered)")
axes[1].set_xlim(-11, 11)
axes[1].set_ylim(-11, 11)
fig.colorbar(im2, ax=axes[1], label="Flux (nJy)", fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()


In [ ]:
# Norms for dynamic range
cutout_norm = simple_norm(cutout_data, stretch='log', percent=99.5)
psf_norm = simple_norm(selectedSource_scaledPsf, stretch='log', percent=99.5)

# Determine PSF coordinates for centering
ny, nx = selectedSource_scaledPsf.shape
x_psf = np.arange(-nx//2, nx//2)
y_psf = np.arange(-ny//2, ny//2)

# Create figure with 2 subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns

# ----- Subplot 1: Visit Image Cutout -----
im1 = axes[0].imshow(
    cutout_data,
    origin='lower',
    norm=psf_norm,
    cmap='viridis',
    extent=[x_grid.min(), x_grid.max(), y_grid.min(), y_grid.max()]
)
axes[0].set_title("Visit Image Cutout")
axes[0].set_xlabel("x (pixels, centered)")
axes[0].set_ylabel("y (pixels, centered)")
axes[0].grid(False)
axes[0].set_xlim(-11, 11)
axes[0].set_ylim(-11, 11)
fig.colorbar(im1, ax=axes[0], label="Flux (nJy)", fraction=0.046, pad=0.04)

# ----- Subplot 2: LSST PSF -----
im2 = axes[1].imshow(
    selectedSource_scaledPsf,
    extent=[x_psf[0], x_psf[-1], y_psf[0], y_psf[-1]],
    origin='lower',
    norm=psf_norm,
    cmap='viridis'
)
axes[1].set_title("LSST PSF (scaled)")
axes[1].set_xlabel("x (pixels, centered)")
axes[1].set_ylabel("y (pixels, centered)")
axes[1].set_xlim(-11, 11)
axes[1].set_ylim(-11, 11)
fig.colorbar(im2, ax=axes[1], label="Flux (nJy)", fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()


In [ ]:
# Use the same normalization across all images for comparison
norm = simple_norm(core_psf, stretch='log', percent=99.5)

fig, axes = plt.subplots(1, 4, figsize=(24, 6))

# -----------------------------
# 1. Visit Image Cutout
ny, nx = cutout_data.shape
extent_cutout = [-(nx-1)/2, (nx-1)/2, -(ny-1)/2, (ny-1)/2]
im0 = axes[0].imshow(cutout_data, origin='lower', norm=norm, cmap='Blues', extent=extent_cutout)
axes[0].set_title("Visit Image Cutout")
axes[0].set_xlabel("X (pixels)")
axes[0].set_ylabel("Y (pixels)")
axes[0].set_xlim(-10, 10)
axes[0].set_ylim(-10, 10)
plt.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04, label="Intensity")

# -----------------------------
# 2. Core PSF
ny, nx = core_psf.shape
extent_core = [-(nx-1)/2, (nx-1)/2, -(ny-1)/2, (ny-1)/2]
im1 = axes[1].imshow(core_psf, origin='lower', norm=norm, cmap='Blues', extent=extent_core)
axes[1].set_title("Core PSF")
axes[1].set_xlabel("X (pixels)")
axes[1].set_ylabel("Y (pixels)")
axes[1].set_xlim(-10, 10)
axes[1].set_ylim(-10, 10)
plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04, label="Intensity")

# -----------------------------
# 3. Extended Wings
ny, nx = wings_ext.shape
extent_wings = [-(nx-1)/2, (nx-1)/2, -(ny-1)/2, (ny-1)/2]
im2 = axes[2].imshow(wings_ext, origin='lower', norm=norm, cmap='Blues', extent=extent_wings)
axes[2].set_title("Extended Wings")
axes[2].set_xlabel("X (pixels)")
axes[2].set_ylabel("Y (pixels)")
axes[2].set_xlim(-10, 10)
axes[2].set_ylim(-10, 10)
plt.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04, label="Intensity")

# -----------------------------
# 4. Core + Smooth Extended Wings
ny, nx = psf_ext.shape
extent_full = [-(nx-1)/2, (nx-1)/2, -(ny-1)/2, (ny-1)/2]
im3 = axes[3].imshow(psf_ext, origin='lower', norm=norm, cmap='Blues', extent=extent_full)
axes[3].set_title("Core + Extended Wings")
axes[3].set_xlabel("X (pixels)")
axes[3].set_ylabel("Y (pixels)")
axes[3].set_xlim(-10, 10)
axes[3].set_ylim(-10, 10)
plt.colorbar(im3, ax=axes[3], fraction=0.046, pad=0.04, label="Intensity")

plt.tight_layout()
plt.show()


In [ ]:
# --- Step 1: Determine padding amounts ---
cutout_shape = cutout_data.shape  # e.g., (51, 51)
psf_shape = core_psf.shape        # e.g., (25, 25)

pad_y = (cutout_shape[0] - psf_shape[0]) // 2
pad_x = (cutout_shape[1] - psf_shape[1]) // 2

# --- Step 2: Pad core PSF and extended wings ---
core_psf_pad = np.pad(core_psf, ((pad_y, pad_y), (pad_x, pad_x)), mode='constant')
psf_ext_pad = np.pad(psf_ext, ((pad_y, pad_y), (pad_x, pad_x)), mode='constant')

# --- Step 3: Compute differences ---
diff_core = cutout_data - core_psf_pad
diff_full = cutout_data - psf_ext_pad

# --- Step 4: Set up visualization ---
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# Common normalization for PSF images
norm_psf = simple_norm(core_psf_pad, stretch='log', percent=99.5)

# Core PSF
extent = [-(cutout_shape[1]-1)/2, (cutout_shape[1]-1)/2,
          -(cutout_shape[0]-1)/2, (cutout_shape[0]-1)/2]
im0 = axes[0,0].imshow(core_psf_pad, origin='lower', norm=norm_psf, cmap='Blues', extent=extent)
axes[0,0].set_title("Core PSF (padded)")
axes[0,0].set_xlabel("X (pixels)")
axes[0,0].set_ylabel("Y (pixels)")
plt.colorbar(im0, ax=axes[0,0], fraction=0.046, pad=0.04)

# Extended Wings
im1 = axes[0,1].imshow(wings_ext, origin='lower', norm=norm_psf, cmap='Blues', extent=extent)
axes[0,1].set_title("Extended Wings")
axes[0,1].set_xlabel("X (pixels)")
axes[0,1].set_ylabel("Y (pixels)")
plt.colorbar(im1, ax=axes[0,1], fraction=0.046, pad=0.04)

# Core + Extended Wings
im2 = axes[0,2].imshow(psf_ext_pad, origin='lower', norm=norm_psf, cmap='Blues', extent=extent)
axes[0,2].set_title("Core + Smooth Extended Wings")
axes[0,2].set_xlabel("X (pixels)")
axes[0,2].set_ylabel("Y (pixels)")
plt.colorbar(im2, ax=axes[0,2], fraction=0.046, pad=0.04)

# Visit cutout
norm_cutout = simple_norm(cutout_data, stretch='log', percent=99.5)
im3 = axes[1,0].imshow(cutout_data, origin='lower', norm=norm_cutout, cmap='viridis', extent=extent)
axes[1,0].set_title("Visit Cutout")
axes[1,0].set_xlabel("X (pixels)")
axes[1,0].set_ylabel("Y (pixels)")
plt.colorbar(im3, ax=axes[1,0], fraction=0.046, pad=0.04)

# Visit - Core PSF
diff_norm = simple_norm(np.abs(diff_core), stretch='log', percent=99.5)
im4 = axes[1,1].imshow(diff_core, origin='lower', norm=diff_norm, cmap='RdBu', extent=extent)
axes[1,1].set_title("Visit - Core PSF")
axes[1,1].set_xlabel("X (pixels)")
axes[1,1].set_ylabel("Y (pixels)")
plt.colorbar(im4, ax=axes[1,1], fraction=0.046, pad=0.04)

# Visit - Core + Extended Wings
diff_norm_full = simple_norm(np.abs(diff_full), stretch='log', percent=99.5)
im5 = axes[1,2].imshow(diff_full, origin='lower', norm=diff_norm_full, cmap='RdBu', extent=extent)
axes[1,2].set_title("Visit - Core + Smooth Extended Wings")
axes[1,2].set_xlabel("X (pixels)")
axes[1,2].set_ylabel("Y (pixels)")
plt.colorbar(im5, ax=axes[1,2], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()


In [ ]:
core_psf = selectedSource_scaledPsf

# Step 1: FWHM and r_half measurement
def fit_fwhm_from_psf(psf_image):
    ny, nx = psf_image.shape
    cy, cx = (ny-1)/2, (nx-1)/2
    y, x = np.indices(psf_image.shape)
    r = np.sqrt((x-cx)**2 + (y-cy)**2)
    r_flat = r.flatten()
    i_flat = psf_image.flatten()

    # radial bins
    r_bins = np.arange(0, min(nx, ny)/2 + 1)
    r_centers = 0.5*(r_bins[:-1] + r_bins[1:])
    prof = np.zeros_like(r_centers)
    for i in range(len(r_centers)):
        mask = (r_flat >= r_bins[i]) & (r_flat < r_bins[i+1])
        prof[i] = np.mean(i_flat[mask]) if np.any(mask) else np.nan
    good = np.isfinite(prof)
    r_centers, prof = r_centers[good], prof[good]
    peak = prof[0]
    half_max = 0.5*peak
    f_interp = interp1d(prof[::-1], r_centers[::-1], bounds_error=False, fill_value="extrapolate")
    r_half = float(f_interp(half_max))
    fwhm = 2.0 * r_half
    return fwhm, r_half

fwhm, r_half = fit_fwhm_from_psf(core_psf)
print(f"FWHM = {fwhm:.3f} pixels, r_half = {r_half:.3f} pixels")

# Step 2: Transition radius and wings generation
def measure_transition_intensity(psf_image, r_transition, dr=1.0):
    ny, nx = psf_image.shape
    cy, cx = (ny-1)/2, (nx-1)/2
    y, x = np.indices(psf_image.shape)
    r = np.sqrt((x-cx)**2 + (y-cy)**2)
    mask = (r >= r_transition - dr/2) & (r <= r_transition + dr/2)
    return np.mean(psf_image[mask])

def generate_psf_with_smooth_wings(core_psf, r_transition, alpha=3.0, I_min_frac=1e-6, smooth_scale=1.0):
    ny, nx = core_psf.shape
    I_transition = measure_transition_intensity(core_psf, r_transition)
    r_max = r_transition * (1/I_min_frac)**(1/alpha)
    r_max = int(np.ceil(r_max))
    
    # Determine extended size
    pad_y = int(np.ceil(r_max - (ny-1)/2))
    pad_x = int(np.ceil(r_max - (nx-1)/2))
    ny_ext, nx_ext = ny + 2*pad_y, nx + 2*pad_x
    cy_ext, cx_ext = (ny_ext-1)/2, (nx_ext-1)/2
    
    y, x = np.indices((ny_ext, nx_ext))
    r = np.sqrt((x - cx_ext)**2 + (y - cy_ext)**2)

    # Wings only
    wings_ext = np.zeros((ny_ext, nx_ext))
    mask_wings = r >= r_transition
    wings_ext[mask_wings] = I_transition * (r[mask_wings]/r_transition)**(-alpha)

    # Core in extended array
    psf_core_ext = np.zeros_like(wings_ext)
    psf_core_ext[pad_y:pad_y+ny, pad_x:pad_x+nx] = core_psf

    # Smooth blending
    blend = 1 / (1 + np.exp(-(r - r_transition)/smooth_scale))
    psf_ext = (1 - blend)*psf_core_ext + blend*wings_ext

    return psf_ext, wings_ext

r_transition = 1.75 * r_half
smooth_scale = 2.0
alpha = 3.0
psf_ext, wings_ext = generate_psf_with_smooth_wings(core_psf, r_transition, alpha=alpha, smooth_scale=smooth_scale)

# Step 3: Pad arrays to cutout size
def pad_to_cutout(psf_array, cutout_shape):
    psf_shape = psf_array.shape
    pad_y = (cutout_shape[0] - psf_shape[0]) // 2
    pad_x = (cutout_shape[1] - psf_shape[1]) // 2
    return np.pad(psf_array, ((pad_y, pad_y), (pad_x, pad_x)), mode='constant')

def match_cutout_size(psf_array, cutout_shape):
    """
    Return an array the same shape as cutout_data.
    Pads with zeros if smaller, crops the center if larger.
    """
    psf_shape = psf_array.shape
    ny, nx = psf_shape
    cy, cx = (ny-1)//2, (nx-1)//2
    cut_ny, cut_nx = cutout_shape
    cut_cy, cut_cx = (cut_ny-1)//2, (cut_nx-1)//2

    # If psf is smaller → pad
    pad_y = max(0, cut_ny - ny)
    pad_x = max(0, cut_nx - nx)
    psf_padded = np.pad(psf_array, ((pad_y//2, pad_y - pad_y//2),
                                    (pad_x//2, pad_x - pad_x//2)), mode='constant')

    # If psf is larger → crop
    psf_padded = psf_padded[
        (psf_padded.shape[0]//2 - cut_cy):(psf_padded.shape[0]//2 + cut_ny - cut_cy),
        (psf_padded.shape[1]//2 - cut_cx):(psf_padded.shape[1]//2 + cut_nx - cut_cx)
    ]
    return psf_padded


cutout_shape = cutout_data.shape
core_psf_pad = match_cutout_size(core_psf, cutout_shape)
psf_ext_pad  = match_cutout_size(psf_ext, cutout_shape)
wings_ext_pad = match_cutout_size(wings_ext, cutout_shape)

# Step 4: Compute residuals
diff_core = cutout_data - core_psf_pad
diff_full = cutout_data - psf_ext_pad

# Step 5: Visualization
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
extent = [-(cutout_shape[1]-1)/2, (cutout_shape[1]-1)/2,
          -(cutout_shape[0]-1)/2, (cutout_shape[0]-1)/2]

norm_psf = simple_norm(core_psf_pad, stretch='log', percent=99.5)
norm_cutout = simple_norm(cutout_data, stretch='log', percent=99.5)

# Top row
axes[0,0].imshow(core_psf_pad, origin='lower', norm=norm_psf, cmap='Blues', extent=extent)
axes[0,0].set_title("Core PSF (padded)")
axes[0,1].imshow(wings_ext_pad, origin='lower', norm=norm_psf, cmap='Blues', extent=extent)
axes[0,1].set_title("Extended Wings")
axes[0,2].imshow(psf_ext_pad, origin='lower', norm=norm_psf, cmap='Blues', extent=extent)
axes[0,2].set_title("Core + Smooth Wings")

# Bottom row
axes[1,0].imshow(cutout_data, origin='lower', norm=norm_cutout, cmap='viridis', extent=extent)
axes[1,0].set_title("Visit Cutout")
diff_norm = simple_norm(np.abs(diff_core), stretch='log', percent=99.5)
axes[1,1].imshow(diff_core, origin='lower', norm=diff_norm, cmap='RdBu', extent=extent)
axes[1,1].set_title("Visit - Core PSF")
diff_norm_full = simple_norm(np.abs(diff_full), stretch='log', percent=99.5)
axes[1,2].imshow(diff_full, origin='lower', norm=diff_norm_full, cmap='RdBu', extent=extent)
axes[1,2].set_title("Visit - Core + Smooth Wings")

for ax in axes.flatten():
    ax.set_xlabel("X (pixels)")
    ax.set_ylabel("Y (pixels)")
plt.tight_layout()
plt.show()

In [ ]:
# Match PSF to cutout size
def match_cutout_size(psf_array, cutout_shape):
    """
    Return an array the same shape as cutout_data.
    Pads with zeros if smaller, crops the center if larger.
    """
    psf_shape = psf_array.shape
    ny, nx = psf_shape
    cy, cx = (ny-1)//2, (nx-1)//2
    cut_ny, cut_nx = cutout_shape
    cut_cy, cut_cx = (cut_ny-1)//2, (cut_nx-1)//2

    # Pad if smaller
    pad_y = max(0, cut_ny - ny)
    pad_x = max(0, cut_nx - nx)
    psf_padded = np.pad(psf_array, ((pad_y//2, pad_y - pad_y//2),
                                    (pad_x//2, pad_x - pad_x//2)), mode='constant')

    # Crop if larger
    psf_padded = psf_padded[
        (psf_padded.shape[0]//2 - cut_cy):(psf_padded.shape[0]//2 + cut_ny - cut_cy),
        (psf_padded.shape[1]//2 - cut_cx):(psf_padded.shape[1]//2 + cut_nx - cut_cx)
    ]
    return psf_padded

cutout_shape = cutout_data.shape

# Step 1: Match PSFs to cutout size
core_psf_pad  = match_cutout_size(core_psf, cutout_shape)
psf_ext_pad   = match_cutout_size(psf_ext, cutout_shape)
wings_ext_pad = match_cutout_size(wings_ext, cutout_shape)

# Step 2: Compute residuals
diff_core = cutout_data - core_psf_pad
diff_full = cutout_data - psf_ext_pad

# Symmetric color limits for differences
diff_max = max(np.abs(diff_core).max(), np.abs(diff_full).max())

# Step 3: Visualization
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

extent = [-(cutout_shape[1]-1)/2, (cutout_shape[1]-1)/2,
          -(cutout_shape[0]-1)/2, (cutout_shape[0]-1)/2]

# --- Row 0: PSFs ---
norm_psf = simple_norm(core_psf_pad, stretch='log', percent=99.5)

im0 = axes[0,0].imshow(core_psf_pad, origin='lower', norm=norm_psf, cmap='Blues', extent=extent)
axes[0,0].set_title("Core PSF")
plt.colorbar(im0, ax=axes[0,0], fraction=0.046, pad=0.04, label="Flux (nJy)")

im1 = axes[0,1].imshow(wings_ext_pad, origin='lower', norm=norm_psf, cmap='Blues', extent=extent)
axes[0,1].set_title("Extended Wings")
plt.colorbar(im1, ax=axes[0,1], fraction=0.046, pad=0.04, label="Flux (nJy)")

im2 = axes[0,2].imshow(psf_ext_pad, origin='lower', norm=norm_psf, cmap='Blues', extent=extent)
axes[0,2].set_title("Core + Smooth Wings")
plt.colorbar(im2, ax=axes[0,2], fraction=0.046, pad=0.04, label="Flux (nJy)")

# --- Row 1: Data + residuals ---
norm_cutout = simple_norm(cutout_data, stretch='log', percent=99.5)
im3 = axes[1,0].imshow(cutout_data, origin='lower', norm=norm_cutout, cmap='Blues', extent=extent)
axes[1,0].set_title("Visit Cutout")
plt.colorbar(im3, ax=axes[1,0], fraction=0.046, pad=0.04, label="Flux (nJy)")

im4 = axes[1,1].imshow(diff_core, origin='lower', vmin=-diff_max, vmax=diff_max, cmap='RdBu', extent=extent)
axes[1,1].set_title("Visit - Core PSF")
plt.colorbar(im4, ax=axes[1,1], fraction=0.046, pad=0.04, label="Flux (nJy)")

im5 = axes[1,2].imshow(diff_full, origin='lower', vmin=-diff_max, vmax=diff_max, cmap='RdBu', extent=extent)
axes[1,2].set_title("Visit - Core + Wings")
plt.colorbar(im5, ax=axes[1,2], fraction=0.046, pad=0.04, label="Flux (nJy)")

for ax in axes.flatten():
    ax.set_xlabel("X (pixels)")
    ax.set_ylabel("Y (pixels)")

plt.tight_layout()
plt.show()


In [ ]:
print("Visit center:", cutout_data[cutout_ny//2, cutout_nx//2])
print("Core PSF center:", core_psf_pad[cutout_ny//2, cutout_nx//2])
print("Core + Wings PSF center:", psf_ext_pad[cutout_ny//2, cutout_nx//2])
